In [38]:
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from sharadar.pipeline.factors import Exchange, Sector, IsDomesticCommonStock, MarketCap, Fundamentals, EV
from sharadar.pipeline.engine import symbol, symbols, make_pipeline_engine
from zipline.pipeline.filters import StaticAssets
import time
import datetime
from zipline.pipeline.factors import AverageDollarVolume
import os
from sharadar.util.output_dir import get_output_dir
from sharadar.pipeline.universes import UniverseReader
from sharadar.pipeline.engine import symbol, symbols, load_sharadar_bundle
import numpy as np
from scipy import stats
from zipline.pipeline.hooks.progress import ProgressHooks

In [31]:
spe = make_pipeline_engine()
pipe_date = pd.to_datetime('2020-11-20', utc=True)

In [40]:
class NullProgressPublisher(object):

    def publish(self, model):
        pass
    
no_hooks = [ProgressHooks.with_static_publisher(NullProgressPublisher())]

In [49]:
pipe_columns = {
    'Close': USEquityPricing.close.latest,
    'sector': Sector(),
    'adv': AverageDollarVolume(window_length=200),
    'mkt_cap': MarketCap()
}

In [32]:
def StocksUS():
    return (
        (USEquityPricing.close.latest > 3) &
        Exchange().element_of(['NYSE', 'NASDAQ', 'NYSEMKT']) &
        (Sector().notnull()) &
        (~Sector().element_of(['Financial Services', 'Real Estate'])) &
        (IsDomesticCommonStock().eq(1)) &
        (Fundamentals(field='revenue_arq') > 0) &
        (Fundamentals(field='assets_arq') > 0) &
        (Fundamentals(field='equity_arq') > 0) &
        (EV() > 0)
    )

In [50]:
pipe = Pipeline(columns=pipe_columns, screen = StocksUS())

stocks_us = spe.run_pipeline(pipe, pipe_date, hooks=no_hooks)
stocks_us.shape

(2218, 4)

In [51]:
min_percentile_mc = stats.percentileofscore(stocks_us['mkt_cap'], 350e6)
min_percentile_adv = stats.percentileofscore(stocks_us['adv'], 2.5e6)

min_percentile_mc, min_percentile_adv

(20.378719567177637, 21.055004508566277)

In [52]:
def TradableStocksUS_Var():
    return (
        (StocksUS()) &
        (AverageDollarVolume(window_length=200).percentile_between(min_percentile_adv, 100.0, mask=StocksUS())) &
        (MarketCap().percentile_between(min_percentile_mc, 100.0, mask=StocksUS()))
    )

pipe = Pipeline(columns=pipe_columns, screen = TradableStocksUS_Var())

stocks_tradable_var = spe.run_pipeline(pipe, pipe_date, hooks=no_hooks)
stocks_tradable_var.shape

/home/c.cerbo/zipline/lib/python3.6/site-packages/zipline-1.4.1-py3.6-linux-x86_64.egg/zipline/pipeline/filters/filter.py:473: RuntimeWarning: invalid value encountered in less_equal
  return (lower_bounds <= data) & (data <= upper_bounds)


(1656, 4)

In [53]:
def TradableStocksUS_Fix():
    return (
        (StocksUS()) &
        (AverageDollarVolume(window_length=200) > 2.5e6) &
        (MarketCap() > 350e6)
    )

pipe = Pipeline(columns=pipe_columns, screen = TradableStocksUS_Fix())

stocks_tradable_fix = spe.run_pipeline(pipe, pipe_date, hooks=no_hooks)
stocks_tradable_fix.shape

(1656, 4)

In [56]:
min_percentile = 20
def TradableStocksUS_Var_20():
    return (
        (StocksUS()) &
        (AverageDollarVolume(window_length=200).percentile_between(min_percentile, 100, mask=StocksUS())) &
        (MarketCap().percentile_between(min_percentile, 100, mask=StocksUS()))
    )

pipe = Pipeline(columns=pipe_columns, screen = TradableStocksUS_Var_20())

stocks_tradable_var20 = spe.run_pipeline(pipe, pipe_date, hooks=no_hooks)
stocks_tradable_var20.shape

/home/c.cerbo/zipline/lib/python3.6/site-packages/zipline-1.4.1-py3.6-linux-x86_64.egg/zipline/pipeline/filters/filter.py:473: RuntimeWarning: invalid value encountered in less_equal
  return (lower_bounds <= data) & (data <= upper_bounds)


(1675, 4)